# DeepSeek fallacy classification

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [2]:
from transformers import EvalPrediction

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_macro = f1_score(labels, predictions, average="macro")
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
        "f1": f1
    }

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch
import torch.nn as nn
from transformers import BitsAndBytesConfig

# Configuración de cuantización 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # NF4 recomendado para QLoRA
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True  # Opcional: nested quantization
)

# Cargar el modelo original como AutoModelForCausalLM
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Asignamos un pad token para que el modelo pueda manejar batch_size > 1
tokenizer.pad_token = tokenizer.eos_token

# Cargar el modelo de lenguaje causal
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)

class ModifiedModelForMulticlassClassification(nn.Module):
    def __init__(self, original_model, num_classes=6):
        super().__init__()
        self.transformer = original_model.model
        self.lm_head = nn.Linear(self.transformer.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state

        # Pooling: media a lo largo de la secuencia
        pooled_output = hidden_states.mean(dim=1)

        # Asegurar que el dtype coincida con el de lm_head
        pooled_output = pooled_output.to(dtype=self.lm_head.weight.dtype)

        logits = self.lm_head(pooled_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.lm_head.out_features), labels.view(-1))
            return loss, logits

        return logits


# Reemplazar el modelo original con el modelo modificado
model = ModifiedModelForMulticlassClassification(model, num_classes=6)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ModifiedModelForMulticlassClassification(
  (transformer): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )

In [46]:
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# Cargar el tokenizer y asignar el pad_token
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Asignar el pad_token como el eos_token

df = pd.read_csv("data/train_afc.csv")
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["Etiqueta"], random_state=42)
train_df = train_df.rename(columns={"Etiqueta": "labels"})
val_df = val_df.rename(columns={"Etiqueta": "labels"})

# Crear la función de tokenización con el prompt
def tokenize_function(examples):
    prompt = """
    Your task is to classify a fallacy in the given text into one of the following categories (numerical labels):
    
    0: Appeal to Emotion
    
    1: Appeal to Authority

    2: Ad Hominem

    3: False Cause

    4: Slippery Slope

    5: Slogans
    
    Task: Classify the following text snippet into one of the fallacy categories (0-5). The text is:
    """
    
    # Concatenate the prompt with the dataset text
    text_with_prompt = [prompt + text for text in examples["Texto"]]
    return tokenizer(text_with_prompt, padding="max_length", truncation=True, max_length=256)


# Tokenizar los datos
train_dataset = Dataset.from_pandas(train_df).map(tokenize_function, batched=True)
val_dataset = Dataset.from_pandas(val_df).map(tokenize_function, batched=True)

# Establecer el formato adecuado para el dataset
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/982 [00:00<?, ? examples/s]

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

In [47]:
from tqdm import tqdm  # Asegúrate de importar esto al principio de tu código
from torch.utils.data import DataLoader

# Establecer el modelo en modo evaluación
model.eval()

# Crear un DataLoader para la inferencia
dataloader = DataLoader(val_dataset, batch_size=8)

predictions = []

# Barra de progreso con tqdm
with torch.no_grad():
    for batch in tqdm(dataloader, desc="Inferencia", total=len(dataloader)):  # Aquí va la barra
        # Mover tensores al dispositivo (GPU o CPU)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Asegurarse de que los input_ids sean del tipo correcto (Long) y no float16
        input_ids = input_ids.to(dtype=torch.long)  # Convertir input_ids a Long
        attention_mask = attention_mask.to(dtype=torch.bfloat16)  # Mantener la máscara como float16 si es necesario

        # Realizar inferencia
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs

        # Obtener las predicciones
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        predictions.extend(preds)

# Guardar las predicciones en el dataframe
val_df['predicted_label'] = predictions


Inferencia: 100%|██████████| 31/31 [00:37<00:00,  1.19s/it]


In [48]:
from sklearn.metrics import accuracy_score, f1_score

# Calcula accuracy y F1 score
accuracy = accuracy_score(val_df['labels'], val_df['predicted_label'])
f1 = f1_score(val_df['labels'], val_df['predicted_label'], average='weighted')
f1_macro = f1_score(val_df['labels'], val_df['predicted_label'], average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"F1 Macro Score: {f1_macro:.4f}")

Accuracy: 0.2236
F1 Score: 0.2312
F1 Macro Score: 0.0921


In [49]:
from datasets import concatenate_datasets

# Concatenar los datasets de entrenamiento y validación
full_train_df = concatenate_datasets([train_dataset, val_dataset]).map(tokenize_function, batched=True)
full_train_df.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/1228 [00:00<?, ? examples/s]

In [51]:
# Cargar test
test_df = pd.read_csv("data/test_afc.csv")
test_df = test_df.rename(columns={"Etiqueta": "labels"})

# Tokenizar igual que los demás
test_dataset = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Crear DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=8)

# Inference
model.eval()
predictions = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Inferencia test", total=len(test_dataloader)):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        input_ids = input_ids.to(dtype=torch.long)
        attention_mask = attention_mask.to(dtype=torch.bfloat16)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs

        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        predictions.extend(preds)

# Guardar resultados
test_df["predicted_label"] = predictions


Map:   0%|          | 0/2160 [00:00<?, ? examples/s]

Inferencia test: 100%|██████████| 270/270 [05:27<00:00,  1.21s/it]


In [52]:
test_df.predicted_label.value_counts()

predicted_label
2    2029
0     115
1      16
Name: count, dtype: int64

In [53]:
test_df.to_csv("afc_deepseak_text.csv", index=False)